In [37]:
import pandas as pd
import numpy as np
import json
import datetime
from datetime import date
from math import radians, sin, cos, sqrt, atan2
from geopy import distance

In [2]:
df = pd.read_csv("projects.csv")

In [3]:
df.head()

,project_name,category,affiliated_agency,description,project_start_time,project_completion_time,total_budget,completion_percentage,location_coordinates
0,JICA Support Program 3 for Strengthening Mathe...,Education,"Ministry of Primary and Mass Education, Direct...",National level of academic skills and knowledg...,2019-04-01,2023-06-30,JPY 380M,77.36%,"(23.729211164246585, 90.40874895549243),(23.80..."
1,Project for Capacity Building of Nursing Servi...,Health,Directorate General of Nursing and Midwifery S...,The quality of nursing education is improved i...,2022-03-26,2026-03-25,JPY 300M,22.15%,"(23.768773179764562, 90.37269632665758),(23.73..."
2,The Project for Strengthening Health Systems t...,Health,Operational Plans (OPs) of the Ministry of Hea...,The Health status of population in Bangladesh ...,2017-06-29,2022-07-28,JPY 400M,97.55%,"(23.728881264793493, 90.40888399782175)"
3,Safe Motherhood Promotion Project,Health,"Ministry of Health and Family Welfare*2, Direc...",Approaches of Reproductive Health (RH) service...,2006-07-01,2010-06-30,JPY 350M,100.00%,"(23.728881264793493, 90.40888399782175),(23.75..."
4,Safe Motherhood Promotion Project Phase 2,Health,"Ministry of Health and Family Welfare*2, Direc...",Maternal and neonatal health status is improve...,2011-07-01,2016-06-30,JPY 420M,100.00%,"(23.728881264793493, 90.40888399782175),(23.75..."


In [4]:
number_of_project, number_of_attribute = df.shape[0], df.shape[1]

def project_lists():
    project_info = []
    
    for id in range(number_of_project):
        loc = df["location_coordinates"][id].split("),")
        locations = np.array([]);
        
        for str in loc:
            str = str.replace("(", "")
            str = str.replace(")", "")
            str = str.replace(" ", "")
            #location = str.split(",")
            location = str
            locations = np.append(locations, np.array(location))
            
        info = {
            "id": id + 1,
            "project_name": df["project_name"][id],
            "location_coordinates": locations
        }
        project_info.append(info)
        
    return project_info
    
print(project_lists()[2]["id"])

3


In [5]:
def project_info_by_id(id): # id is 1 index based
    return df.iloc[id - 1] if 0 < id <= number_of_project else "Project not found!"
    
print(type(project_info_by_id(1)["project_start_time"]))

<class 'str'>


In [6]:
def project_filter_based_on_category(category):
    projects = project_lists();
    after_filter = [project for project in projects if project_info_by_id(project["id"])["category"] == category]
    
    return after_filter

print(project_filter_based_on_category("Health"))
    

[{'id': 2, 'project_name': 'Project for Capacity Building of Nursing Services Phase 2', 'location_coordinates': array(['23.768773179764562,90.37269632665758',
       '23.733211657612223,90.40993638432778'], dtype='<U36')}, {'id': 3, 'project_name': 'The Project for Strengthening Health Systems through Organizing Communities', 'location_coordinates': array(['23.728881264793493,90.40888399782175'], dtype='<U36')}, {'id': 4, 'project_name': 'Safe Motherhood Promotion Project', 'location_coordinates': array(['23.728881264793493,90.40888399782175',
       '23.75363622259384,90.39417243785537',
       '23.780215725696586,90.40895332665768'], dtype='<U36')}, {'id': 5, 'project_name': 'Safe Motherhood Promotion Project Phase 2', 'location_coordinates': array(['23.728881264793493,90.40888399782175',
       '23.75363622259384,90.39417243785537',
       '23.780215725696586,90.40895332665768'], dtype='<U36')}]


In [7]:
def string_to_date(str):
    str = str.split("-")
    return date(int(str[0]), int(str[1]), int(str[2]))

def project_filter_based_on_timeframe(daterange):
    dates = daterange.split(":")
    if len(dates) < 2:
        dates.append("") 
        
    start = string_to_date(dates[0]) if dates[0] != "" else date(1, 1, 1)
    end = string_to_date(dates[1]) if dates[1] != "" else date(9999, 12, 31)
        
    projects = project_lists()
    after_filter = [project for project in projects if string_to_date(project_info_by_id(project["id"])["project_start_time"]) >= start and string_to_date(project_info_by_id(project["id"])["project_completion_time"]) <= end]
    return after_filter
    
#project_filter_based_on_timeframe("2014-12-17:2022-12-31")
project_filter_based_on_timeframe(":2022-12-31")

[{'id': 3,
  'project_name': 'The Project for Strengthening Health Systems through Organizing Communities',
  'location_coordinates': array(['23.728881264793493,90.40888399782175'], dtype='<U36')},
 {'id': 4,
  'project_name': 'Safe Motherhood Promotion Project',
  'location_coordinates': array(['23.728881264793493,90.40888399782175',
         '23.75363622259384,90.39417243785537',
         '23.780215725696586,90.40895332665768'], dtype='<U36')},
 {'id': 5,
  'project_name': 'Safe Motherhood Promotion Project Phase 2',
  'location_coordinates': array(['23.728881264793493,90.40888399782175',
         '23.75363622259384,90.39417243785537',
         '23.780215725696586,90.40895332665768'], dtype='<U36')},
 {'id': 6,
  'project_name': 'National Integrity Strategy Support Project Phase 2',
  'location_coordinates': array(['23.728407931193587,90.40787482665709'], dtype='<U36')},
 {'id': 7,
  'project_name': 'Project for Capacity Development of City Corporations (C4C)',
  'location_coordinate

In [8]:
def time_remaining(id): #percentage
    project = project_info_by_id(id)
    start, curr, end = string_to_date(project["project_start_time"]), date.today(), string_to_date(project["project_completion_time"])
    return ((end - curr) / (end - start)) * 100
    
time_remaining(1)

12.508059316569955

In [48]:
def calculate_geo_distance_Haversine_formula(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = radians(lat1), radians(lon1), radians(lat2), radians(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    radius = 6371
    return radius * c

lat1 = 52
lon1 = 21
lat2 = 21
lon2 = 105
dist = calculate_geo_distance_Haversine_formula(lat1, lon1, lat2, lon2)
print(f"Distance between the points: {dist:.2f} km")

Distance between the points: 7780.54 km


In [57]:
def calculate_geo_distance_Vincenty_formula(lat1, lon1, lat2, lon2): 
    return distance.distance((lat1, lon1), (lat2, lon2)).meters

calculate_geo_distance_Vincenty_formula(lat1, lon1, lat2, lon2)

7791560.480953166